<a href="https://colab.research.google.com/github/zahere-dev/basic-ai-agent/blob/main/zahere_basic_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests

In [2]:
  # agent.py
import requests
import json
import ast

class Agent:
    def __init__(self, api_key):
        self.api_key = api_key
        self.tools = []
        self.memory = []
        self.max_memory = 5

    def add_tool(self, tool: Tool):
        self.tools.append(tool)

    def json_parser(self, input_string):
      python_dict = ast.literal_eval(input_string)
      json_string = json.dumps(python_dict)
      json_dict = json.loads(json_string)

      if isinstance(json_dict, dict):
        return json_dict

      raise "Invalid JSON response"

    def process_input(self, user_input):
        self.memory.append(f"User: {user_input}")
        self.memory = self.memory[-self.max_memory:]

        context = "\n".join(self.memory)
        tool_descriptions = "\n".join([f"- {tool.name()}: {tool.description()}" for tool in self.tools])
        response_format = {"action":"", "args":""}

        prompt = f"""Context:
        {context}

        Available tools:
        {tool_descriptions}

        Based on the user's input and context, decide if you should use a tool or respond directly.
        Sometimes you might have to use multiple tools to solve user's input. You have to do that in a loop.
        If you identify a action, respond with the tool name and the arguments for the tool.
        If you decide to respond directly to the user then make the action "respond_to_user" with args as your response in the following format.

        Response Format:
        {response_format}

        """

        response = self.query_llm(prompt)
        self.memory.append(f"Agent: {response}")

        response_dict = self.json_parser(response)

        # Check if any tool can handle the input
        for tool in self.tools:
            if tool.name().lower() == response_dict["action"].lower():
                return tool.use(response_dict["args"])

        return response_dict

    def query_llm(self, prompt):
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        data = {
            "model": "gpt-4o-mini-2024-07-18",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 150
        }
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))
        final_response = response.json()['choices'][0]['message']['content'].strip()
        print("LLM Response ", final_response)
        return final_response

NameError: name 'Tool' is not defined

In [ ]:
import datetime
import requests
from pytz import timezone
from abc import ABC, abstractmethod

class Tool(ABC):
    @abstractmethod
    def name(self) -> str:
        pass

    @abstractmethod
    def description(self) -> str:
        pass

    @abstractmethod
    def use(self, *args, **kwargs):
        pass

class TimeTool(Tool):
    def name(self):
        return "Time Tool"

    def description(self):
        return "Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time."

    def use(self, *args, **kwargs):
        format = "%Y-%m-%d %H:%M:%S %Z%z"
        current_time = datetime.datetime.now()
        input_timezone = args[0]
        if input_timezone:
            print("TimeZone", input_timezone)
            current_time =  current_time.astimezone(timezone(input_timezone))
        return f"The current time is {current_time}."


class WeatherTool(Tool):
    def name(self):
        return "Weather Tool"

    def description(self):
        return "Provides weather information for a given location"

    def use(self, *args, **kwargs):
        location = args[0].split("weather in ")[-1]
        api_key = userdata.get("OPENWEATHERMAP_API_KEY")
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        response = requests.get(url)
        data = response.json()
        if data["cod"] == 200:
            temp = data["main"]["temp"]
            description = data["weather"][0]["description"]
            return f"The weather in {location} is currently {description} with a temperature of {temp}°C."
        else:
            return f"Sorry, I couldn't find weather information for {location}."

In [ ]:
  # main.py

from google.colab import userdata

def main():
    api_key = userdata.get("OPENAI_API_KEY")
    agent = Agent(api_key)

    # Add tools to the agent
    agent.add_tool(TimeTool())
    agent.add_tool(WeatherTool())

    print("LLM Agent: Hello! How can I assist you today?")

    user_input = input("You: ")
    while True:
      if user_input.lower() in ["exit", "bye", "close"]:
        print("See you later!")
        break
      response = agent.process_input(user_input)
      if isinstance(response, dict) and response["action"] == "respond_to_user":
        print("Reponse from Agent: ", response["args"])
        user_input = input("You: ")
      else:
        user_input = response

if __name__ == "__main__":
    main()